In [49]:
import pandas as pd, json, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
cluster=json.loads(file('../json/cluster.json','r').read())
citysave=json.loads(file('../json/citysave.json','r').read())

In [51]:
import wolframalpha
app_id='A4TU9P-E3QAGQJL39'
#dcsala@masdar.ac.ae
client = wolframalpha.Client(app_id)

In [52]:
unicities={}
for i in cluster:
    if cluster[i] not in unicities:
        unicities[cluster[i]]=citysave[i]['country']

In [53]:
G={}
error=[]

In [8]:
uk=unicities.keys()
for h in range(900,1800):
    c=uk[h]
    if c not in G.keys()+error:
        print h,
        ys={"pop":0,"nearby":{}}
        q='population of '+c+', '+unicities[c]
        try:
            res = client.query(q)
            for i in range(len(res['pod'])):
                if res['pod'][i]['@title']=="Result":
                    x=res['pod'][i]['subpod']['plaintext']
                    popul=x[:x.find('people')-1]
                    if 'mill' in popul:
                            popul=float(popul[:popul.find('mill')-1])*1000000.0
                    ys['pop']=int(popul)
                if res['pod'][i]['@title']=="Nearby cities":
                    x=res['pod'][i]['subpod']['plaintext'].split('\n')
                    for y in x[:-1]:
                        people=y[y.rfind('|')+2:y.find('people')-1]
                        if 'mill' in people:
                            people=float(people[:people.find('mill')-1])*1000000.0
                        km=float(y[y.find('|')+2:y.find('km')-1])
                        ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
            G[c]=ys
        except: error.append(c)

900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055 1056 1057 1058 1059 1060 1061 1062 1063 1064 1065 1066 1067 1068 1069 1070 1071 1072 1073 1074 1075 1076 1077 1078 1079 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 1090 1091 1092 1093 1094 1095 1096 1097 1098 1099 1100 1101 1102 1103 1104 1105 1106 1107 1108 1109 1110 1111 1112 1113 1114 1115 1116 1117 1118 1119 

In [13]:
file("../json/pop2.json",'w').write(json.dumps(G))
file("../json/pop2e.json",'w').write(json.dumps(error))

Postprocessing

In [54]:
G=json.loads(file('../json/pop2.json','r').read())
error=json.loads(file('../json/pop2e.json','r').read())

In [55]:
error2=[]

In [56]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

In [ ]:
for c in error:
    if c not in G.keys()+error2:
        ys={"pop":0,"nearby":{}}
        q=remove_accents(strip_accents('population of '+c.split('/')[0]+', '+unicities[c]))
        res = client.query(q)
        if 'pod' in res:
            for i in range(len(res['pod'])):
                if res['pod'][i]['@title']=="Result":
                    x=res['pod'][i]['subpod']['plaintext']
                    if 'available' not in x:
                        popul=x[:x.find('people')-1]
                        if 'mill' in popul:
                            popul=popul[:popul.find('mill')-1]
                            if '|' in popul:popul=popul.split('|')[1].strip()
                        ys['pop']=int(float(popul)*1000000.0)
                if res['pod'][i]['@title']=="Nearby cities":
                    x=res['pod'][i]['subpod']['plaintext'].split('\n')
                    if 'available' not in x:
                        for y in x[:-1]:
                            people=y[y.rfind('|')+2:y.find('people')-1]
                            if 'mill' in people:
                                people=float(people[:people.find('mill')-1])*1000000.0
                            km=float(y[y.find('|')+2:y.find(' km ')])
                            ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
                        G[c]=ys
                        print 'success',c
        else: 
            print 'error',c
            error2.append(c)

In [19]:
file("../json/pop2b.json",'w').write(json.dumps(G))
file("../json/pop2eb.json",'w').write(json.dumps(error2))
print len(G),len(error),len(error2)

729 655 69


Re-reparse

In [57]:
G=json.loads(file('../json/pop2b.json','r').read())
error=json.loads(file('../json/pop2eb.json','r').read())

In [58]:
error3=[]
error4=[]
uk=unicities.keys()
for h in range(900,1800):
    c=uk[h]
    if c not in G:
        error3.append(c)
print len(error3)

171


In [67]:
for c in error3:
    if c not in G.keys()+error4:
        ys={"pop":0,"nearby":{}}
        q=remove_accents(strip_accents('population of '+c.split('/')[0]+', '+unicities[c]))
        res = client.query(q)
        good=True
        if 'pod' in res:
            for i in range(len(res['pod'])):
                try:
                    if res['pod'][i]['@title']=="Result":
                        x=res['pod'][i]['subpod']['plaintext']
                        if 'available' not in x:
                            popul=x[:x.find('people')-1]
                            if 'mill' in popul:
                                popul=popul[:popul.find('mill')-1]
                                if '|' in popul:popul=popul.split('|')[1].strip()
                            ys['pop']=int(float(popul)*1000000.0)
                    if res['pod'][i]['@title']=="Nearby cities":
                        x=res['pod'][i]['subpod']['plaintext'].split('\n')
                        if 'available' not in x:
                            for y in x[:-1]:
                                people=y[y.rfind('|')+2:y.find('people')-1]
                                if 'mill' in people:
                                    people=float(people[:people.find('mill')-1])*1000000.0
                                km=float(y[y.find('|')+2:y.find(' km ')])
                                ys['nearby'][y.split('|')[0].split(',')[0].strip()]={"km":km,"people":int(people)}
                            G[c]=ys
                            print 'success',c
                            good=False
                except: pass
        if good: 
            print 'error',c
            error4.append(c)

error Ethiopia
error Matei
error Nevis
error Colville Lake
error Mont-Joli
error Samburu South
error Kebar
error Kingfisher Lake
error Australia
error Larantuka
error Djibouti
error Malawi
error Yangyang
error Lebanon
error Kastelorizo Island
error Anglesey
error South Andros
error Islay
error Zhezkazgan
error Songpan
error Saudi Arabia
error Tawitawi
error Mahshahr
error Dikson
error Tennant Creek
error Barter Island
error Atka
error Pajala
error Bermuda
error Peru
error Benin
error Virgin Gorda
error Castro
error San Cristobal
error Vanua Balavu
error Pangnirtung
error Bamaga
error Fuerteventura
error Palm Island
error Charleville
error Churchill
error Graciosa
error Isle of Man
error Iron Mountain Kingsford
error Qinhuangdao
error Dobo-Kobror Island
error Muan
error Barra
error Formosa
error Longyearbyen
error Hachijojima
error Rankin Inlet
error Lawas
error Long Datih
error Roti-Rote Island
error Puerto Jimenez
error Lansdowne House
error Barimunya
error Marudi
error Mirny
error Is

In [68]:
file("../json/pop2c.json",'w').write(json.dumps(G))
file("../json/pop2ec.json",'w').write(json.dumps(error4))
print len(G),len(error4)

730 170
